In [1]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

In [2]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        super(tutorial_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(in_features=400, out_features=180)
        self.fc2 = nn.Linear(in_features=180, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=80)
        self.fc4 = nn.Linear(in_features=80, out_features=10)
        
    def forward(self, x):
        """ Chain all layers together """
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 400)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)



In [3]:
#Number of epochs for training
num_epochs = 2

# Batch Size for training/testing
batch_size = 32

# Learning Rate for optimizer
learning_rate = 0.005

# Dimensions of CIFAR10
dim = 32

# Momentum for Optimizer
momentum = 0.5


In [4]:
#Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim, scale=(0.7, 1.0), ratio=(1.0,1.0)),
    transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.CenterCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [5]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

Files already downloaded and verified


In [6]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (SGD)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)  

# Set model to training (updating weights)
model.train();

In [9]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/2, Step: 1/1563, Loss: 2.0907061100006104, Accuracy: 37.5%
Epoch: 1/2, Step: 101/1563, Loss: 2.187824249267578, Accuracy: 21.875%
Epoch: 1/2, Step: 201/1563, Loss: 1.825063943862915, Accuracy: 28.125%
Epoch: 1/2, Step: 301/1563, Loss: 1.9340792894363403, Accuracy: 34.375%
Epoch: 1/2, Step: 401/1563, Loss: 1.9687609672546387, Accuracy: 18.75%
Epoch: 1/2, Step: 501/1563, Loss: 1.9722671508789062, Accuracy: 28.125%
Epoch: 1/2, Step: 601/1563, Loss: 1.9770240783691406, Accuracy: 25.0%
Epoch: 1/2, Step: 701/1563, Loss: 1.789726734161377, Accuracy: 21.875%
Epoch: 1/2, Step: 801/1563, Loss: 2.018716335296631, Accuracy: 25.0%
Epoch: 1/2, Step: 901/1563, Loss: 1.9376310110092163, Accuracy: 25.0%
Epoch: 1/2, Step: 1001/1563, Loss: 1.7933963537216187, Accuracy: 40.625%
Epoch: 1/2, Step: 1101/1563, Loss: 1.8378682136535645, Accuracy: 31.25%
Epoch: 1/2, Step: 1201/1563, Loss: 1.8138394355773926, Accuracy: 25.0%
Epoch: 1/2, Step: 1301/1563, Loss: 1.6533515453338623, Accuracy: 37.5%
Epoch: 1

In [10]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 41.160000000000004%
